In [ ]:
import secrets, json, re, pathlib, getpass
 
##
## Made by Petr Nebukin, based on Zin script.
##
 
# Installing SSH server
!apt-get update > /dev/null
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server > /dev/null

!read -t 1 -p "SSH server installed!"
!echo -e "\n"

#Installing Apache2
!apt install apache2
!sudo /etc/init.d/apache2 start

!read -t 1 -p "Apache2 installed!"
!echo -e "\n"
 
# Installing ngrok
!wget -nv -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n ngrok-stable-linux-amd64.zip
!sudo cp ngrok /bin

!read -t 1 -p "Ngrok downloaded!"
!echo -e "\n"
 
# Creating user accounts
root_password = "toor"
user_password = "mitm"
user_name = "petr"
!useradd -s /bin/bash -m $user_name
!echo root:$root_password | chpasswd
!echo $user_name:$user_password | chpasswd
!usermod -aG sudo $user_name
!mkdir -p /run/sshd
!service ssh restart
 
# Setting up ngrok
if not pathlib.Path('/root/.ngrok2/ngrok.yml').exists():
  !./ngrok authtoken 1l1BEBvtKyAJ3Exyg4HlsYVdJbn_251bGPcQKRKPFGmiu3o19
 
region = "eu"

get_ipython().system_raw("./ngrok tcp -region {} 22 &".format(region))
!read -t 1 -p "Opened port 22 (ssh)"
tunnels = !curl -s http://localhost:4040/api/tunnels
url = json.loads(tunnels[0])["tunnels"][0]["public_url"]
m = re.match("tcp://(.+):(\d+)", url)
hostname = m.group(1)
port = m.group(2)
ssh_common_options = "-o UserKnownHostsFile=/dev/null -o ServerAliveInterval=60"

 
# Give the user login credentials etc.
print("-"*24)
print("root password: {}".format(root_password))
print("{} password: {}".format(user_name, user_password))
print("---")
print("Command to connect to the ssh server:")
print("ssh {} -p {} {}@{}".format(ssh_common_options, port, user_name, hostname))
print("---")
print("To open HTTP port via ngrok:")
print("1. Start SSH session with server")
print("2. Type [ngrok http 80] and press Enter")
print("3. Close SSH session")


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:KPZLYl7U0Vl0ySwtebnib0CmWlBSdFqAHHKwvguw5eE root@bf20e0df639d (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:5P5G3k3TqsScKeRJyM8eNT9xlKQ51Sm5GcTM/Nb5cvg root@bf20e0df639d (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:pldUfjw3ciO7VzPPkZxsW4ehJhkZk5MoWnPWGZsG5Z4 root@bf20e0df639d (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
SSH server installed!

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to r

In [ ]:
!bash